# ***Milestone 1 - Data Sanity ( by using Numpy and Pandas)***

Impoorting Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sea
import numpy as np
import scipy as sc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn import svm
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Function for data modification

In [ ]:
def difference_in_days(melt, lags, ffday, customer_id_col, create_date_col, net_amount_col):
    for i in range(ffday, lags+1):
        melt['Last-'+str(i)+'day_Sales'] = melt.groupby([customer_id_col])[net_amount_col].shift(i)
    
    melt = melt.reset_index(drop = True)
    
    for i in range(ffday, lags+1):
        melt['Last-'+str(i)+'day_Diff']  = melt.groupby([customer_id_col])['Last-'+str(i)+'day_Sales'].diff()
    melt = melt.fillna(0)
    return melt

1. Use the PRS dataset to create a dataframe

In [ ]:
harsh=pd.read_csv("Final.csv")
display(harsh)

In [ ]:
type(harsh)

2. Check the description of the dataframe

In [ ]:
harsh.describe()

3. Check the shape of the dataframe

In [ ]:
harsh.shape

4. Check the data frame informations

In [ ]:
harsh.info()

5. Check for the Null values in the dataframe

In [ ]:
harsh.isnull().sum()

6. Replace all the null values with "NaN"

In [ ]:
harsh=harsh.fillna("NaN")
harsh.isnull().sum()

7. Change the format of date columns - "ORDER_CREATION_DATE" to datetime[64] with the format as "%Y%m%d"

In [ ]:
harsh['ORDER_CREATION_DATE'] = pd.to_datetime(harsh['ORDER_CREATION_DATE'], format='%Y%m%d')
display(harsh)

8. Do the same activity for the other date field i.e. "REQUESTED_DELIVERY_DATE" to datetime[64] with the format as "%Y%m%d"

In [ ]:
harsh['REQUESTED_DELIVERY_DATE'] = pd.to_datetime(harsh['REQUESTED_DELIVERY_DATE'], format='%Y%m%d')
display(harsh)

9. Sanity check - Check how many records are having order date greater than the delivery date

In [ ]:
a=(harsh['ORDER_CREATION_DATE']>harsh['REQUESTED_DELIVERY_DATE']).sum()
print(a)

10. Remove those records where order date is greater than the delivery date 

In [ ]:
a=harsh['ORDER_CREATION_DATE']>harsh['REQUESTED_DELIVERY_DATE']
harsh=harsh.loc[~a]
display(harsh)

11. Check the number of records where the “ORDER_AMOUNT” field is having “-” in it.

In [ ]:
a=(harsh['ORDER_AMOUNT'].str.contains('-')).sum()
print(a)

12. Replace “-” with “” from the “ORDER_AMOUNT” field. 

In [ ]:
harsh['ORDER_AMOUNT']=harsh['ORDER_AMOUNT'].str.replace("-",'')
harsh['RELEASED_CREDIT_VALUE']=harsh['RELEASED_CREDIT_VALUE'].str.replace("-",'')
display(harsh)

13. Check the number of records where the “ORDER_AMOUNT” field is having “,” in it..

In [ ]:
a=(harsh['ORDER_AMOUNT'].str.contains(',')).sum()
print(a)

14. Replace “,” with “.” from the “ORDER_AMOUNT” field. 

In [ ]:
harsh['ORDER_AMOUNT']=harsh['ORDER_AMOUNT'].str.replace(",",'.')
display(harsh)

15. Count the number of records where the order date and the delivery date are same

In [ ]:
a=(harsh['ORDER_CREATION_DATE']==harsh['REQUESTED_DELIVERY_DATE']).sum()
print(a)

16. Count the number of records for each currency type by using the field “'ORDER_CURRENCY'”

In [ ]:
a=(harsh.groupby('ORDER_CURRENCY')).count()
display(a)

17. Create a new column in the existing dataframe as “'amount_in_usd'” and convert all the non-USD currencies in USD and store them in the same column. 

harsh['amount_in_usd'] = harsh.apply(lambda x: float(x['ORDER_AMOUNT']) * float(dic[x['ORDER_CURRENCY']]), axis=1)

In [ ]:
exchange_rates = {
    'USD':1,                                                     
    'EUR': 1.08,   
    'AUD': 0.65,    
    'CAD': 0.74,    
    'GBP': 1.24,    
    'MYR': 0.22,    
    'PLN': 0.24,    
    'AED': 0.27,   
    'HKD': 0.13,     
    'CHF': 1.11,     
    'RON': 0.22,     
    'SGD': 0.74,     
    'CZK': 0.045,     
    'HU1': 0.0029,     
    'NZD': 0.61,       
    'BHD': 2.65,      
    'SAR': 0.27,       
    'QAR': 0.27,       
    'KWD': 3.25,       
    'SEK': 0.094
}
harsh['amount_in_usd'] = harsh['ORDER_AMOUNT'].astype(float) * harsh['ORDER_CURRENCY'].map(exchange_rates)

In [ ]:
display(harsh)

18. Check for values “0” in the “'amount_in_usd” column.

In [ ]:
a=(harsh['amount_in_usd']==0).sum()
print(a)

19. Create a new column in the existing dataframe “unique_cust_id” by adding 'CUSTOMER_NUMBER' and 'COMPANY_CODE'

In [ ]:
harsh["unique_cust_id"]=str(harsh['CUSTOMER_NUMBER']) + str(harsh['COMPANY_CODE'])
display(harsh)

# ***Milestone 2 - EDA***


1. Create a Histogram on DISTRIBUTION_CHANNEL

In [ ]:
plt.subplots(figsize=(20,10))
plt.hist(harsh['DISTRIBUTION_CHANNEL'].sample(n=1000),color='orange',bins=100)
#title of the histogram
plt.title('Distribution of DISTRIBUTION_CHANNEL')
plt.xlabel('DISTRIBUTION_CHANNEL',)
plt.ylabel('COUNT')
plt.xticks([])
plt.show()

In [ ]:
plt.subplots(figsize=(20,10))
sea.histplot(harsh['DISTRIBUTION_CHANNEL'].sample(n=1000),color='orange',bins=100) #taking sample of 1000 randomly
plt.xticks([])
plt.xlabel('DISTRIBUTION_CHANNEL')

In [ ]:
#distribution channel names are messing up with each other in histogram so I am removing it in order to make clear view
dist_channel=harsh['DISTRIBUTION_CHANNEL'].value_counts()[:25]
dist_others=harsh['DISTRIBUTION_CHANNEL'].value_counts()[25:].sum()
dist_channel['others'] = dist_others
arr=[]
for i in range(1,27):
    arr.append(i)
newdfDist=pd.DataFrame(dist_channel)
newdfDist['numbercountry']=arr

#plotting histogram
newdfDist.hist(figsize=[15,7])
plt.show()

2. Create a Pie Chart on ORDER_CURRENCY

In [ ]:
plt.subplots(figsize=(15,7))
plt.pie(harsh['ORDER_CURRENCY'].value_counts(), labels=harsh['ORDER_CURRENCY'].unique(),autopct='%1.0f%%')

# Add a legend
plt.legend(loc = 'center left',fontsize=6)

# Show the pie chart
plt.show()


From pie chart it can be inferred that EUR currency is mosty being order currency

3. Create a line chart PURCHASE_ORDER_TYPE and DISTRIBUTION_CHANNEL

In [ ]:
#'value' must be an instance of str or bytes, not a float so converting it into list 
harsh_array = harsh['PURCHASE_ORDER_TYPE'].tolist()
plt.plot(harsh_array, harsh['DISTRIBUTION_CHANNEL'],'o-', linewidth=2)
# Set the title and labels
plt.title('PURCHASE_ORDER_TYPE vs DISTRIBUTION_CHANNEL')
plt.xlabel('PURCHASE_ORDER_TYPE')
plt.ylabel('DISTRIBUTION_CHANNEL')
plt.yticks([])
plt.xticks([])
plt.show()

In [ ]:
#Seeing above it is looking messy so , i am taking sample of it and then proceeding
sample_df = harsh.sample(n=100, random_state=42) 
plt.subplots(figsize=(15,7))
df_array = sample_df['PURCHASE_ORDER_TYPE'].tolist()
plt.plot(sample_df['DISTRIBUTION_CHANNEL'],df_array,'og-', linewidth=2)
plt.title('PURCHASE_ORDER_TYPE vs DISTRIBUTION_CHANNEL')
plt.xlabel('DISTRIBUTION_CHANNEL')
plt.ylabel('PURCHASE_ORDER_TYPE')
plt.yticks([]) #removing the label values because it was messing with other values and making visual poor.
plt.xticks([])
plt.show()

In [ ]:
# Both are categorical value so we can go for separate line plots which is done below separaterly
# for PURCHASE_ORDER_TYPE
purchaseOrderType=harsh['PURCHASE_ORDER_TYPE'].value_counts()[:20]
purchaseOthers=harsh['PURCHASE_ORDER_TYPE'].value_counts()[20:].sum()
purchaseOrderType['others'] = purchaseOthers
newDfPurchase = pd.DataFrame(purchaseOrderType)
#standardizing the values
from sklearn.preprocessing import StandardScaler
  
# define standard scaler
scaler = StandardScaler()
  
# transform data
newdfStand= scaler.fit_transform(newDfPurchase)
newdfStand = pd.DataFrame(newdfStand)
newDfPurchase['Standardized Data']=newdfStand[0].astype(float).values

#plotting
newDfPurchase.plot.line()
plt.show()

In [ ]:
#for distribution channel
##DISTRIBUTION_CHANNEL
dist_chnl=harsh['DISTRIBUTION_CHANNEL'].value_counts()[:21]
others_chnl=harsh['DISTRIBUTION_CHANNEL'].value_counts()[21:].sum()
dist_chnl['others'] = others_chnl
#create new dataframe
newdfChnl=pd.DataFrame(dist_chnl)
newdfChnl.plot.line()
plt.show()

4. Create a line plot on ORDER_CREATION_DATE and amount_in_usd



In [ ]:
plt.subplots(figsize=(15,7))
# Create the line chart
plt.plot(harsh['ORDER_CREATION_DATE'], harsh['amount_in_usd'], 'o-b', linewidth=2)

# Set the title and labels
plt.title('ORDER_CREATION_DATE vs amount_in_usd')
plt.xlabel('ORDER_CREATION_DATE')
plt.ylabel('amount_in_usd')
# Show the chart
plt.show()

In [ ]:
# Prepare the sample data
sample_df = harsh.sample(n=1000)
plt.subplots(figsize=(15,7))
# Convert 'ORDER_AMOUNT' to numeric type
sample_df['amount_in_usd'] = pd.to_numeric(sample_df['ORDER_AMOUNT'], errors='coerce')

# Group data by 'ORDER_CREATION_DATE' and calculate the sum of 'amount_in_usd'
grouped_data = sample_df.groupby('ORDER_CREATION_DATE')['amount_in_usd'].sum()

# Create the line plot
grouped_data.plot(kind='line',color='brown')

# Set labels and title
plt.xlabel('Order Creation Date')
plt.ylabel('Amount in USD')
plt.title('Line Plot of Order Creation Date and Amount in USD')


In [ ]:
#it looks bad so lets do it separately 
#for ORDER_CREATION_DATE attribute
order=harsh['ORDER_CREATION_DATE'].value_counts()[:25]
others_order=harsh['ORDER_CREATION_DATE'].value_counts()[25:].sum()
order['others'] = others_order
order.plot.line(figsize=[15,7])
plt.xlabel('ORDER_CREATION_DATE')
plt.show()

In [ ]:
#for amount_in_usd attribute
amount=harsh['amount_in_usd'].value_counts()
others_amount=harsh['amount_in_usd'].value_counts().sum()
amount['others'] = others_amount
amount.plot.line(figsize=[15,7])
plt.xlabel('amount_in_usd')
plt.show()

5. Create a boxplot on ORDER_AMOUNT	to find out the outliers

In [ ]:
#converting type to float so that it will do the calculation because it is in string format
order_amounts = harsh['ORDER_AMOUNT'].astype(float)
plt.subplots(figsize=(15,7))
plt.boxplot(order_amounts, 0, 'gs', 0)
plt.show()

In [ ]:
newdf = pd.DataFrame(order_amounts)
plt.figure(figsize=(15,7))
sea.boxplot(data=newdf, showfliers=False, saturation=1,color = 'skyblue')
plt.title("Boxplot for Order Amount", fontsize = 25)
plt.xlabel("Indices", size=12)
plt.ylabel('Order Amount', size=12)
plt.xticks(fontsize=20)
plt.show()

6. Create a barchart on COMPANY_CODE and ORDER_AMOUNT

In [ ]:
#using sample dataframe
plt.subplots(figsize=(15,7))
sample_df = harsh.sample(n=10000, random_state=42) 
ax = sea.barplot(x='ORDER_AMOUNT', y='COMPANY_CODE', data=sample_df)
plt.title("Barchart of ordered amount and company code  ")
plt.xticks(rotation=90) 
plt.xticks([]) #all x-axis label value were messing so i removed it to look pretty
plt.show()

In [ ]:
#Above chart is to messy so in order to view it clearly taking only 20 samples
plt.subplots(figsize=(15,7))
sample_df = harsh.sample(n=20, random_state=42) 
ax = sea.barplot(x='ORDER_AMOUNT', y='COMPANY_CODE', data=sample_df)
plt.title("Barchart of ordered amount and company code  ")
plt.xticks(rotation=90) 
 #all x-axis label value were messing so i removed it to look pretty
plt.show()

In [ ]:
#To visualize in proper way we can use some another method by visualizing both features separately
newDataFrame=harsh['COMPANY_CODE'].value_counts()[:32]
others=harsh['COMPANY_CODE'].value_counts()[32:].sum()
newDataFrame['others']=others
newDataFrame.plot(kind='bar',figsize=[15,7],xlabel='Company Code',ylabel='Counts',rot=0)
plt.show()

In [ ]:
plt.subplots(figsize=(15,7))
newDataFrame1=harsh['ORDER_AMOUNT'].value_counts()[:20]
others1=harsh['ORDER_AMOUNT'].value_counts()[20:].sum()
newDataFrame1['others'] = others1
newDataFrame1.plot(kind='bar',figsize=[15,7],xlabel='Order Amount',ylabel='Counts',rot=0,width=0.5)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(harsh['ORDER_AMOUNT'].astype('float64'))
plt.ylabel('ORDER_AMOUNT')
plt.title('Boxplot of ORDER_AMOUNT to find outliers')
plt.show()

In [ ]:
plt.figure(figsize=(40,6))
sns.boxplot(x=harsh['amount_in_usd'])
plt.title('Boxplot')
plt.show()

# Milestone 3 - Feature Engineering and Selection


1. Check for the outliers in the “amount_in_usd” column and replace the outliers with appropriate values, discussed in the sessions.


In [ ]:
Q1=harsh["amount_in_usd"].quantile(0.25)
Q2=harsh["amount_in_usd"].median()
Q3=harsh["amount_in_usd"].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - (1.5 * IQR)
upper_bound = Q3 + (1.5 * IQR)

# Identify outliers
outliers = harsh[(harsh['amount_in_usd'] < lower_bound) | (harsh['amount_in_usd'] > upper_bound)]
harsh.loc[(harsh['amount_in_usd'] < lower_bound) | (harsh['amount_in_usd'] > upper_bound), 'amount_in_usd'] = Q2

In [ ]:
outliers.count()

In [ ]:
## Now finding by using Standard deviation

mean_value = harsh['amount_in_usd'].mean()
std_value = harsh['amount_in_usd'].std()

# Define the threshold for outliers (e.g., 3 standard deviations)
threshold = 1

# Identify outliers
outliers1 = harsh[(harsh['amount_in_usd'] - mean_value).abs() > threshold * std_value]


In [ ]:
outliers1.count()

2. Label encoding or One hot Encoding on all the categorical columns 

In [ ]:
display(harsh)

In [ ]:
#converting datatypes into float 
harsh["ORDER_AMOUNT"]=harsh["ORDER_AMOUNT"].astype(float)
harsh['RELEASED_CREDIT_VALUE']=harsh['RELEASED_CREDIT_VALUE'].str.replace(",",'.')
harsh["RELEASED_CREDIT_VALUE"]=harsh["RELEASED_CREDIT_VALUE"].astype(float)
harsh["CREDIT_STATUS"]=harsh["CREDIT_STATUS"].astype(float)
harsh=harsh.convert_dtypes()

In [ ]:
harsh.info()

In [ ]:

# Separate categorical and numerical columns
cat=[]
num=[]
for i in harsh.columns:
  if harsh[i].dtype=="string" :
    cat.append(i)
  elif harsh[i].dtype=="Int64" or harsh[i].dtype=="Float64":
    num.append(i)

ans=cat+num

#changeing datatype of catagorical values string to catagory
harsh[cat]=harsh[cat].astype('category')
harsh.info()

In [ ]:

# Label encode categorical columns
le = LabelEncoder()
harsh[cat] = harsh[cat].apply(le.fit_transform)


3. Log Transformations on continuous columns 

In [ ]:
harsh[num] = np.log(harsh[num])
display(harsh)

4. Try to extract new features by grouping existing columns 

In [ ]:
# I have not grouped any one as I don't require it

5. Create a heatmap to find correlation between the columns

In [ ]:
plt.subplots(figsize=(15,7))
corr_matrix=harsh.corr()
sea.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
corr_matrix

6. Try to identify important or relevant columns for feature extraction

In [ ]:
corr_matrix=harsh.corr()

threshold = 0.5  # Set the correlation threshold
relevant_columns = []

for col in corr_matrix.columns:
    if abs(corr_matrix['amount_in_usd'][col]) >= threshold:  # Check correlation with the target variable
        relevant_columns.append(col)

print('Relevant columns for feature extraction:', relevant_columns)

# Milestone 4 - ML Models and Evaluations

1. Modify the dataset to pass into any type of machine learning models. 


In [ ]:
#Adding sum new columns to dataset which are relevent for pridiction
harsh=difference_in_days(harsh,5,1,"unique_cust_id","ORDER_CREATION_DATE","amount_in_usd")
display(harsh)


In [ ]:

##Ploting the heatmap after adding new columns 
plt.subplots(figsize=(15,7))
corr_matrix=harsh.corr()
sea.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
#Trying to identify important or relevant columns for feature extraction

threshold = 0.5  # Set the correlation threshold
relevant = []

for col in corr_matrix.columns:
    if abs(corr_matrix['amount_in_usd'][col]) >= threshold:  # Check correlation with the target variable
        relevant.append(col)

print('Relevant columns for feature extraction:', relevant)

In [ ]:
##Here I am dropping RELEASED_CREDIT_VALUE as it has 93% null values
harsh=harsh.drop("RELEASED_CREDIT_VALUE",axis =1)
#Removing this column as this has no use as we are using column "amount_in_usd"
harsh=harsh.drop("ORDER_AMOUNT",axis=1)
#now dropping column containg Date 
harsh=harsh.drop(["ORDER_CREATION_DATE","REQUESTED_DELIVERY_DATE"],axis =1)
#now droping unique_cust_id column as it is not usabele for pridiction
harsh=harsh.drop("unique_cust_id",axis =1)

In [ ]:
harsh = harsh.fillna(harsh.mean())

harsh = harsh.apply(lambda col: col.fillna(col.mean()), axis=1)

display(harsh)



In [ ]:
#replacing infineite value with 0
harsh.replace([np.nan, -np.inf], 0, inplace=True)

#harsh = harsh.apply(lambda col: col.fillna(col.mean()), axis=0)

display(harsh)

In [ ]:
plt.subplots(figsize=(15,7))
corr_matrix=harsh.corr()
sea.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
display(harsh)

2. Try different machine learning models like - 
a Linear Regression
b Support Vector Machine 
c Decision Tree
d Random Forest 
e AdaBoost
f Xgboost etc.


3. Perform Regression model evaluations like MSE, RMSE, R-Square etc.

I have done both the questions together.

In [ ]:
#comman function for every learning model to training and finding it's accuracy
def evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred1 = model.predict(X_train)
    print('Model Performance for {}'.format(model.__class__.__name__))
    print('R2: ', r2_score(y_test, y_pred))
    print('R2 for traning: ', r2_score(y_train, y_pred1))
    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
    print()


In [ ]:
harsh1=harsh[['CREDIT_STATUS',
 'amount_in_usd',
 'Last-1day_Sales',
 'Last-2day_Sales',
 'Last-3day_Sales',
 'Last-4day_Sales',
 'Last-5day_Sales',
 'Last-1day_Diff',
 'Last-2day_Diff',
 'Last-3day_Diff',
 'Last-4day_Diff',
 'Last-5day_Diff']]

harsh1.replace([np.nan, -np.inf], 0, inplace=True)

harsh1 = harsh1.apply(lambda col: col.fillna(col.mean()), axis=1)

display(harsh1)

In [ ]:
has_nan_or_inf = harsh1.isnull().any(axis=1) | harsh1.isin([np.inf, -np.inf]).any(axis=1)

# Count rows with NaN or inf values
numrows = has_nan_or_inf.sum()

print(numrows)

In [ ]:
#Creating training and testing data
x=harsh1.drop('amount_in_usd',axis =1)
y=harsh1["amount_in_usd"]
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=4)

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Your code here

evaluate(LinearRegression(), X_train, y_train, X_test, y_test)


In [ ]:
evaluate(DecisionTreeRegressor(), X_train, y_train, X_test, y_test)

In [ ]:
evaluate(ExtraTreesRegressor(), X_train, y_train, X_test, y_test)

In [ ]:
evaluate(AdaBoostRegressor(), X_train, y_train, X_test, y_test)

In [ ]:
# Import necessary libraries
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

harsh1=harsh[[ 'CREDIT_STATUS','amount_in_usd','Last-1day_Sales','Last-2day_Sales','Last-3day_Sales','Last-4day_Sales','Last-5day_Sales']]
x=harsh1.drop('amount_in_usd',axis =1)
y=harsh1["amount_in_usd"]

#X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=4)

# Create a DMatrix for XGBoost (an internal data structure used by XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical='true')
dtest = xgb.DMatrix(X_test)

# Set hyperparameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # For regression tasks
    'max_depth': 100,  # Maximum depth of each tree
    'eta': 0.1,  # Learning rate
    'subsample': 0.7,  # Fraction of training data used in each boosting round
    'colsample_bytree': 0.9,  # Fraction of features used in each boosting round
    'num_boost_round': 10  # Number of boosting rounds (trees)
}    

# Train the XGBoost regressor
model = xgb.train(params, dtrain)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
r21 = r2_score(y_pred, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared (R2) Score: {r2}")
print(f"R-squared (R2) for test Score: {r21}")

# Plot feature importance
plot_importance(model)
plt.show()


In [ ]:
evaluate(RandomForestRegressor(), X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import numpy as np

# Load your dataset or create X (input features) and y (target variable)

# Create a linear regression model
model = RandomForestRegressor(max_depth=12, max_features='sqrt', max_leaf_nodes=16,n_estimators=150)

# Define the number of folds for cross-validation
k = 5

# Perform k-fold cross-validation
cross_val_scores = cross_val_score(model, x, y, cv=k, scoring='r2')

# Print the R2 scores obtained from each fold
for fold, score in enumerate(cross_val_scores):
    print(f"Fold {fold+1}: R2 Score = {score}")

# Calculate the average R2 score and its standard deviation
avg_score = np.mean(cross_val_scores)
std_dev = np.std(cross_val_scores)

print(f"\nAverage R2 Score: {avg_score}")
print(f"Standard Deviation: {std_dev}")


In [ ]:
evaluate(svm.SVR(), X_train, y_train, X_test, y_test)
#I have executed it but I am not including it as it is too long to train

4. Compare the accuracies of all the models 

5. Select the best possible model


6. Perform Hyperparameter tuning, select best hyperparameters by using appropriate algorithms


In [ ]:
# Here I am using RandomsearchCV for hyperperameter tuning of my model
# Necessary imports

from sklearn.model_selection import RandomizedSearchCV

# Creating the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150,200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10,12,16],
    'max_leaf_nodes': [12,14,16],
    'criterion': ['squared_error'],
    'min_samples_split': [4,5,6],
    'min_samples_leaf': [3,4,5] ,
    'min_weight_fraction_leaf': [0] ,
    'min_impurity_decrease': [1,2,3],
    'bootstrap': ['true'],
    'oob_score': ['true'],
    'n_jobs': [2,3,4],
    'random_state': [20,30,40],
    'verbose': [0],
    'warm_start': ['true'],
    'ccp_alpha': [2,1,4],
    'max_samples': [20,30,40]
}

# Instantiating RandomizedSearchCV object
random_search = RandomizedSearchCV(RandomForestRegressor(),param_grid)
random_search.fit(X_train, y_train)
print(random_search.best_estimator_)


7. Come up with the best possible model accuracy.


In [ ]:
evaluate(ExtraTreesRegressor(bootstrap=True, ccp_alpha=2, max_depth=16,
                    max_features=None, max_leaf_nodes=12, max_samples=30,
                    min_impurity_decrease=2, min_samples_leaf=5,
                    min_samples_split=4, min_weight_fraction_leaf=0, n_jobs=4,
                    oob_score=True, random_state=20, warm_start=True),X_train, y_train, X_test, y_test)

In [ ]:
evaluate(RandomForestRegressor(bootstrap=True, ccp_alpha=2, max_depth=16,
                      max_features='sqrt', max_leaf_nodes=14, max_samples=30,
                      min_impurity_decrease=2, min_samples_leaf=3,
                      min_samples_split=6, min_weight_fraction_leaf=0, n_jobs=4,
                      oob_score=True, random_state=30, warm_start=True),X_train, y_train, X_test, y_test)

In [ ]:
grid_search = GridSearchCV(RandomForestRegressor(),param_grid,cv=5)
grid_search.fit(X_train, y_train)
print(random_search.best_estimator_)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
#As we are getting less accuracy than original regressor we will not use this updated model for training propose

#We will use Random forest for Pridiction as it is having highest accuracy 

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of decision trees in the random forest
    'max_depth': [ 10,12,16],  # Maximum depth of each decision tree
    'min_samples_split': [5, 10,15],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 4 , 8],  # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'log2']  # Maximum number of features to consider when looking for the best split
}


tree = RandomForestRegressor()

# Instantiating RandomizedSearchCV object
grid_search = GridSearchCV(RandomForestRegressor(),param_grid,cv=5)
grid_search.fit(X_train, y_train)
print(random_search.best_estimator_)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

# Load your dataset or create X (input features) and y (target variable)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

# Create the model
model = MLPRegressor(validation_fraction=0.2, n_iter_no_change=10)

# Train the model with early stopping
model.fit(X_train, y_train)

# Evaluate the model's performance on test data or using cross-validation
test_score = model.score(X_test, y_test)
test_score2 = model.score(X_train, y_train)
print(f"Test Score: {test_score}")
print(f"Train Score: {test_score2}")



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

# Load your dataset or create X (input features) and y (target variable)

# Create the model with regularization
model = Ridge(alpha=0.5)

# Define the number of folds for cross-validation
k = 5

# Perform cross-validation
cross_val_scores = cross_val_score(model, x, y, cv=k, scoring='r2')

# Print the average R2 score
avg_score = cross_val_scores.mean()
print(f"Average R2 Score: {avg_score}")


In [ ]:
import pandas as pd

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the DataFrame using the encoder
harsh1 = pd.DataFrame(encoder.fit_transform(harsh[cat]).toarray())

# Get the column names for the one-hot encoded column

# Rename the DataFrame columns
df_encoded.columns = encoded_columns

# Concatenate the original DataFrame with the encoded DataFrame
harsh1 = pd.concat([harsh, harsh1], axis=1)

# Print the final DataFrame
print(harsh1)



In [ ]:
harsh.info()

In [ ]:
model=RandomForestRegressor(bootstrap=True, ccp_alpha=2, max_depth=16,
                      max_features='sqrt', max_leaf_nodes=14, max_samples=30,
                      min_impurity_decrease=2, min_samples_leaf=3,
                      min_samples_split=6, min_weight_fraction_leaf=0, n_jobs=4,
                      oob_score=True, random_state=30, warm_start=True)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred1 = model.predict(X_train)
print('Model Performance for {}'.format(model.__class__.__name__))
print('R2: ', r2_score(y_test, y_pred))
print('R2 for traning: ', r2_score(y_train, y_pred1))
print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
print()

In [ ]:
import pickle

with open('final1_model.sav', 'wb') as model_file:
    pickle.dump(model, model_file)